# Day 9: Explosives in Cyberspace

> The format compresses a sequence of characters. Whitespace is ignored. To indicate that some sequence should be repeated, a marker is added to the file, like (10x2). To decompress this marker, take the subsequent 10 characters and repeat them 2 times. Then, continue reading the file after the repeated data. The marker itself is not included in the decompressed output.

> If parentheses or other characters appear within the data referenced by a marker, that's okay - treat it like normal data, not a marker, and then resume looking for markers after the decompressed section.

## Part 1

And we have to find the decompressed length of the file. I guess the sensible thing to do is to write a generator that returns the uncompressed data, so if part 2 needs to know the details of what's being uncompressed, we can easily get it.

In [1]:
from utils import *

inp = read_input_as_file('day9')
inp = inp.readline()
head(inp)

(6x6


To convert a string to an iterator, which we'd like for the generator, we need to do the following:

In [2]:
inp_iter = inp.__iter__()

Which would make the decompression function:

In [3]:
def decode_marker(s: iter):
    c1 = int(cat(takewhile(lambda c: c != 'x', s)))
    c2 = int(cat(takewhile(lambda c: c != ')', s)))
    return c1, c2

assert decode_marker('10x2)'.__iter__()) == (10, 2)
assert decode_marker('987123x1324234)'.__iter__()) == (987123, 1324234)

    

def decompress(s: str):
    s = s.__iter__()
    length = 0
    while True:
        c = next(s, 'end')
        if c == 'end':
            break
        if c == '(':
            (l, times) = decode_marker(s)
            _ = list(islice(s, l)) # take `l` chars
            # we only care about the length...
            length += (l * times)
        else:
            length += 1
    return length
            
assert decompress('A(1x5)BC') == 7
assert decompress('X(8x2)(3x3)ABCY') == 18

decompress(inp)

150914

Correct! The only real issue in this part was that islice doesn't actually consume the chars unless I tell it to using `list`. I think there's probably a better function for this use case.

## Part 2

> In version two, the only difference is that markers within decompressed data are decompressed.

The question also says:

> Unfortunately, the computer you brought probably doesn't have enough memory to actually decompress the file; you'll have to come up with another way to get its decompressed length.

This is an easy part 2, we just need to make the decompress function recursive.

In [4]:
def decompress(s: str):
    s = s.__iter__()
    length = 0
    while True:
        c = next(s, 'end')
        if c == 'end':
            break
        if c == '(':
            (l, times) = decode_marker(s)
            chars = list(islice(s, l)) # take `l` chars
            # instead of l being the length, we will use the decoded length of `chars`
            length += (decompress(chars) * times)
        else:
            length += 1
    return length

In [11]:
assert decompress('(3x3)XYZ') == 9
assert decompress('X(8x2)(3x3)ABCY') == 20
assert decompress('(27x12)(20x12)(13x14)(7x10)(1x12)A') == 241920
assert decompress('(25x3)(3x3)ABC(2x3)XY(5x2)PQRSTX(18x9)(3x2)TWO(5x7)SEVEN') == 445

In [12]:
decompress(inp)

11052855125